In [1]:
import gymnasium as gym
import torch as th
import mujoco
import stable_baselines3 as sb
import ray

print(gym.__version__)
print(th.__version__)
print(mujoco.__version__)
print(sb.__version__)
print(ray.__version__)

0.29.1
2.5.1+cu118
3.2.4
2.3.2
2.38.0


In [9]:
import gymnasium as gym

# Initialise the environment
env = gym.make("LunarLander-v2", continuous=True, render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(1000):
    # this is where you would insert your policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

TypeError: Converting from sequence to b2Vec2, expected int/float arguments index 0

In [9]:
import gymnasium as gym
import torch as th

from stable_baselines3 import A2C

# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
# Note: an extra linear layer will be added on top of the pi and the vf nets, respectively
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], vf=[64, 64]))
# Create the agent
model = A2C("MlpPolicy", "Pendulum-v1", policy_kwargs=policy_kwargs, verbose=1)
# Retrieve the environment
env = model.get_env()
# Train the agent
model.learn(total_timesteps=100_000)
# Save the agent
model.save("ppo_llv2")

del model
# the policy_kwargs are automatically loaded
model = A2C.load("ppo_llv2", env=env)

Using cuda device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.35e+03 |
| time/                 |           |
|    fps                | 370       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.44     |
|    explained_variance | 0.8979026 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -17       |
|    std                | 1.02      |
|    value_loss         | 62.9      |
-------------------------------------
---------------------------------------
| rollout/              |             |
|    ep_len_mean        | 200         |
|    ep_rew_mean        | -1.36e+03   |
| time

In [12]:
obs, info = env.reset()

In [13]:
obs, info

(array([-0.8635671 ,  0.50423384, -0.71662366], dtype=float32), {})

In [14]:
 action, _states = model.predict(obs, deterministic=True)

In [17]:
print(_states)

None


In [15]:
action

array([2.], dtype=float32)

In [19]:
import gymnasium as gym
from stable_baselines3 import A2C

# Memuat model yang telah dilatih
model = A2C.load("ppo_llv2")

# Inisialisasi lingkungan
env = gym.make("Pendulum-v1", render_mode="human")

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
while not done or truncated:
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=True)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    env.render()

# Menutup lingkungan setelah selesai
env.close()

TypeError: size must be two numbers

In [20]:
env.close()

In [1]:
import gymnasium as gym
import torch as th
from stable_baselines3 import DQN, A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Set konfigurasi umum
env_id = "LunarLander-v2"
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[64, 64, 64], vf=[64, 64]))

# Inisialisasi model untuk setiap algoritma
algorithms = {
    "A2C": A2C("MlpPolicy", env_id, policy_kwargs=policy_kwargs, verbose=0),
    "PPO": PPO("MlpPolicy", env_id, policy_kwargs=policy_kwargs, verbose=0),
}

# Latih setiap model dan simpan hasilnya
trained_models = {}
for algo_name, model in algorithms.items():
    print(f"Training {algo_name}...")
    model.learn(total_timesteps=20_000, progress_bar=True)
    trained_models[algo_name] = model
    model.save(f"{algo_name}_lunarlander")

# Evaluasi performa setiap model
results = {}
for algo_name, model in trained_models.items():
    # Evaluasi model menggunakan evaluate_policy
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    results[algo_name] = {"mean_reward": mean_reward, "std_reward": std_reward}
    print(f"{algo_name}: Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# Print perbandingan hasil
print("\nComparison of results:")
for algo_name, result in results.items():
    print(f"{algo_name}: Mean Reward = {result['mean_reward']}, Std Reward = {result['std_reward']}")


c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Training A2C...


Training PPO...


A2C: Mean reward = -490.11 +/- 318.44
PPO: Mean reward = -217.97 +/- 20.94

Comparison of results:
A2C: Mean Reward = -490.1056629000001, Std Reward = 318.43748785392296
PPO: Mean Reward = -217.97396650000002, Std Reward = 20.938320972389807


In [2]:
import gymnasium as gym
from stable_baselines3 import A2C

# Memuat model yang telah dilatih
model = A2C.load("A2C_lunarlander")

# Inisialisasi lingkungan
env = gym.make("LunarLander-v2", render_mode="human")

# Mulai episode inferensi
obs, info = env.reset()
done = False
truncated = False
while not done or truncated:
    # Pilih tindakan berdasarkan model
    action, _states = model.predict(obs, deterministic=True)
    # Lakukan aksi di lingkungan
    obs, reward, done, truncated, info = env.step(action)
    # Render visualisasi hasilnya
    env.render()

# Menutup lingkungan setelah selesai
env.close()


KeyboardInterrupt: 

In [3]:
env.close()

In [ ]:
import gymnasium as gym

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy


# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Instantiate the agent
model = DQN("MlpPolicy", env, verbose=1)
# Train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)
# Save the agent
model.save("dqn_lunar")
del model  # delete trained model to demonstrate loading

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_lunar", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

# Enjoy trained agent
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [2]:
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [ ]:
vec_env.close()

In [ ]:
!pip install gymnasium-robotics

In [ ]:
!pip install gymnasium[mujoco]==0.29.1

In [ ]:
import gymnasium as gym
import torch as th
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Konfigurasi kebijakan (policy)
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[64, 64])

# Inisialisasi lingkungan MuJoCo 'HalfCheetah-v4'
env_id = "HalfCheetah-v4"
env = gym.make(env_id)

# Inisialisasi model PPO
model = PPO("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=1)

# Latih agen
print("Starting training...")
model.learn(total_timesteps=100_000)

# Simpan model yang sudah dilatih
model.save("ppo_halfcheetah")

# Evaluasi model yang telah dilatih
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

# Hasil evaluasi
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Tutup lingkungan
env.close()


In [10]:
import gymnasium as gym
import torch as th
from stable_baselines3 import A2C, PPO, SAC, TD3, DDPG
from stable_baselines3.common.evaluation import evaluate_policy

# Set environment
env_id = "BipedalWalker-v3"

# Policy configuration for each algorithm
policy_configs = {
    "A2C": dict(activation_fn=th.nn.ReLU, net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    "PPO": dict(activation_fn=th.nn.ReLU, net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    "SAC": dict(activation_fn=th.nn.ReLU, net_arch=[256, 256]),  # Single net for both pi and vf
    "TD3": dict(activation_fn=th.nn.ReLU, net_arch=[256, 256]),  # Larger network for TD3
    "DDPG": dict(activation_fn=th.nn.ReLU, net_arch=[256, 256])  # Deeper network for DDPG
}

# Initialize models with specific policies
algorithms = {
    "A2C": A2C("MlpPolicy", env_id, policy_kwargs=policy_configs["A2C"], verbose=0),
    "PPO": PPO("MlpPolicy", env_id, policy_kwargs=policy_configs["PPO"], verbose=0),
    "SAC": SAC("MlpPolicy", env_id, policy_kwargs=policy_configs["SAC"], verbose=0),
    "TD3": TD3("MlpPolicy", env_id, policy_kwargs=policy_configs["TD3"], verbose=0),
    "DDPG": DDPG("MlpPolicy", env_id, policy_kwargs=policy_configs["DDPG"], verbose=0)
}

# Train each model and save results
trained_models = {}
for algo_name, model in algorithms.items():
    print(f"Training {algo_name}...")
    model.learn(total_timesteps=20_000, progress_bar=True)
    trained_models[algo_name] = model
    model.save(f"{algo_name}_bipedalwalker")

# Evaluate each model
results = {}
for algo_name, model in trained_models.items():
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    results[algo_name] = {"mean_reward": mean_reward, "std_reward": std_reward}
    print(f"{algo_name}: Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# Print comparison of results
print("\nComparison of results:")
for algo_name, result in results.items():
    print(f"{algo_name}: Mean Reward = {result['mean_reward']}, Std Reward = {result['std_reward']}")


Training A2C...


Training PPO...


Training SAC...


Training TD3...


Training DDPG...


A2C: Mean reward = -100.46 +/- 0.29
PPO: Mean reward = -104.64 +/- 6.40
SAC: Mean reward = -123.83 +/- 2.37
TD3: Mean reward = -111.88 +/- 14.77
DDPG: Mean reward = -130.41 +/- 13.93

Comparison of results:
A2C: Mean Reward = -100.4582122, Std Reward = 0.2876229910044744
PPO: Mean Reward = -104.63782269999999, Std Reward = 6.395481108335009
SAC: Mean Reward = -123.82576960000002, Std Reward = 2.374024677732194
TD3: Mean Reward = -111.87758670000001, Std Reward = 14.766777505273453
DDPG: Mean Reward = -130.41362249999997, Std Reward = 13.92941335745438


In [1]:
from pprint import pprint

from ray.rllib.algorithms.ppo import PPOConfig

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .env_runners(num_env_runners=1)
)

algo = config.build()

for i in range(10):
    result = algo.train()
    result.pop("config")
    pprint(result)

    if i % 5 == 0:
        checkpoint_dir = algo.save_to_path()
        print(f"Checkpoint saved in directory {checkpoint_dir}")

c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\rllib\algorithms\algorithm.py:568: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may 

{'date': '2024-10-31_12-52-38',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': 21.56},
                 'episode_duration_sec_mean': 0.010456471003126354,
                 'episode_len_max': 84,
                 'episode_len_mean': 21.56,
                 'episode_len_min': 9,
                 'episode_return_max': 84.0,
                 'episode_return_mean': 21.56,
                 'episode_return_min': 9.0,
                 'module_episode_returns_mean': {'default_policy': 21.56},
                 'num_agent_steps_sampled': {'default_agent': 4000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 4000},
                 'num_env_steps_sampled': 4000,
                 'num_env_steps_sampled_lifetime': 4000,
                 'num_episodes': 185,
                 'num_module_steps_sampled': {'default_policy': 4000},
                 'num_module_steps_sampled_lifetime': {'default_policy': 4000},
                 'sample': 2.1002275

In [2]:
from ray import train, tune

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .training(
        lr=tune.grid_search([0.01, 0.001, 0.0001]),
    )
)

tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"env_runners/episode_return_mean": 150.0},
    ),
)

tuner.fit()

(SingleAgentEnvRunner pid=39316) 2024-10-31 12:54:22,715	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(PPO pid=6504) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/User/ray_results/PPO_2024-10-31_12-54-11/PPO_CartPole-v1_8d5c1_00001_1_lr=0.0010_2024-10-31_12-54-11/checkpoint_000000)
(PPO pid=6504) 2024-10-31 12:54:22,785	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future! [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/maste

2024-10-31 12:55:16,074	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/User/ray_results/PPO_2024-10-31_12-54-11' in 0.0090s.
2024-10-31 12:55:16,611	INFO tune.py:1041 -- Total run time: 64.94 seconds (64.39 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'timers': {'env_runner_sampling_timer': 1.4698971616560688, 'learner_update_timer': 5.683219029279236, 'synch_weights': 0.004406476745286862, 'synch_env_connectors': 0.005091966095070545, 'training_iteration_time_sec': 7.686489582061768, 'restore_workers_time_sec': 0.00016637643178304037, 'training_step_time_sec': 7.685988545417786}, 'env_runners': {'num_agent_steps_sampled_lifetime': {'default_agent': 84000}, 'episode_len_max': 500, 'episode_return_mean': 160.06, 'module_episode_returns_mean': {'default_policy': 160.06}, 'num_agent_steps_sampled': {'default_agent': 4000}, 'num_env_steps_sampled_lifetime': 144000, 'num_env_steps_sampled': 4000, 'episode_len_min': 12, 'episode_duration_sec_mean': 0.12663740099873394, 'episode_len_mean': 160.06, 'num_module_steps_sampled': {'default_policy': 4000}, 'num_episodes': 13, 'agent_episode_returns_mean': {'default_agent': 160.06}, 'num_module_steps_sampled_lifetime': {'default_policy': 84000}, 'sample': np.fl

In [3]:
from ray import train, tune

# Tuner.fit() allows setting a custom log directory (other than ~/ray-results).
tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"num_env_steps_sampled_lifetime": 20000},
        checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True),
    ),
)

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(
    metric="env_runners/episode_return_mean", mode="max"
)

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint

2024-10-31 12:56:01,908	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/User/ray_results/PPO_2024-10-31_12-55-16' in 0.0170s.
2024-10-31 12:56:02,549	INFO tune.py:1041 -- Total run time: 45.85 seconds (45.19 seconds for the tuning loop).


In [ ]:
checkpoint_path = 
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint(checkpoint_path)

NameError: name 'checkpoint_path' is not defined

In [4]:
import pathlib
import gymnasium as gym
import numpy as np
import torch
from ray.rllib.core.rl_module import RLModule

env = gym.make("CartPole-v1")

# Create only the neural network (RLModule) from our checkpoint.
rl_module = RLModule.from_checkpoint(
    pathlib.Path(best_checkpoint.path) / "learner_group" / "learner" / "rl_module"
)["default_policy"]

episode_return = 0
terminated = truncated = False

obs, info = env.reset()

while not terminated and not truncated:
    # Compute the next action from a batch (B=1) of observations.
    torch_obs_batch = torch.from_numpy(np.array([obs]))
    action_logits = rl_module.forward_inference({"obs": torch_obs_batch})[
        "action_dist_inputs"
    ]
    # The default RLModule used here produces action logits (from which
    # we'll have to sample an action or use the max-likelihood one).
    action = torch.argmax(action_logits[0]).numpy()
    obs, reward, terminated, truncated, info = env.step(action)
    episode_return += reward

print(f"Reached episode return of {episode_return}.")

Reached episode return of 500.0.


In [8]:
from ray.rllib.algorithms.ppo import PPOConfig

algo = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .env_runners(num_env_runners=2)
).build()

# Get weights of the algo's RLModule.
algo.get_module().get_state()

# Same as above
algo.env_runner.module.get_state()

# Get list of weights of each EnvRunner, including remote replicas.
algo.env_runner_group.foreach_worker(lambda env_runner: env_runner.module.get_state())

# Same as above, but with index.
algo.env_runner_group.foreach_worker_with_id(
    lambda _id, env_runner: env_runner.module.get_state()
)

c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\rllib\algorithms\algorithm.py:568: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may 

[OrderedDict([('encoder.actor_encoder.net.mlp.0.weight',
               array([[ 0.48074734, -0.27554852,  0.3861252 ,  0.04286963],
                      [-0.24321026,  0.10605699, -0.31143463,  0.45463032],
                      [-0.2310844 ,  0.13257575,  0.40450126,  0.40058738],
                      ...,
                      [-0.39735532, -0.4630162 , -0.48954093, -0.20519793],
                      [-0.01912349,  0.17111182, -0.12658882, -0.05784327],
                      [ 0.3242514 ,  0.21938777, -0.07781363,  0.06862128]],
                     dtype=float32)),
              ('encoder.actor_encoder.net.mlp.0.bias',
               array([-0.42349046, -0.09525579,  0.19313997,  0.24745929,  0.48743117,
                      -0.44139707, -0.24722278, -0.11840814, -0.07465196, -0.15854478,
                      -0.25044543, -0.4148047 , -0.25724047,  0.15085763,  0.44752496,
                      -0.3305989 ,  0.00122595,  0.0494591 , -0.45621443,  0.44512528,
                  

In [9]:
try:
    import gymnasium as gym

    env = gym.make("ALE/Pong-v5")
    obs, infos = env.reset()
except Exception:
    import gym

    env = gym.make("PongNoFrameskip-v4")
    obs = env.reset()

# RLlib uses preprocessors to implement transforms such as one-hot encoding
# and flattening of tuple and dict observations.
from ray.rllib.models.preprocessors import get_preprocessor

prep = get_preprocessor(env.observation_space)(env.observation_space)
# <ray.rllib.models.preprocessors.GenericPixelPreprocessor object at 0x7fc4d049de80>

# Observations should be preprocessed prior to feeding into a model
obs.shape
# (210, 160, 3)
prep.transform(obs).shape
# (84, 84, 3)

(84, 84, 3)

In [10]:
# Get a reference to the policy
import numpy as np
import torch

from ray.rllib.algorithms.dqn import DQNConfig

algo = (
    DQNConfig()
    .api_stack(
        enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False
    )
    .framework("torch")
    .environment("CartPole-v1")
    .env_runners(num_env_runners=0)
    .training(
        replay_buffer_config={
            "type": "MultiAgentPrioritizedReplayBuffer",
        }
    )
).build()
# <ray.rllib.algorithms.ppo.PPO object at 0x7fd020186384>

policy = algo.get_policy()
# <ray.rllib.policy.eager_tf_policy.PPOTFPolicy_eager object at 0x7fd020165470>

# Run a forward pass to get model output logits. Note that complex observations
# must be preprocessed as in the above code block.
logits, _ = policy.model({"obs": torch.from_numpy(np.array([[0.1, 0.2, 0.3, 0.4]]))})
# (<tf.Tensor: id=1274, shape=(1, 2), dtype=float32, numpy=...>, [])

# Compute action distribution given logits
policy.dist_class
# <class_object 'ray.rllib.models.tf.tf_action_dist.Categorical'>
dist = policy.dist_class(logits, policy.model)
# <ray.rllib.models.tf.tf_action_dist.Categorical object at 0x7fd02301d710>

# Query the distribution for samples, sample logps
dist.sample()
# <tf.Tensor: id=661, shape=(1,), dtype=int64, numpy=..>
dist.logp(torch.tensor([1]))
# <tf.Tensor: id=1298, shape=(1,), dtype=float32, numpy=...>

# Get the estimated values for the most recent forward pass
policy.model.value_function()
# <tf.Tensor: id=670, shape=(1,), dtype=float32, numpy=...>

print(policy.model)
"""
Model: "model"
_____________________________________________________________________
Layer (type)               Output Shape  Param #  Connected to
=====================================================================
observations (InputLayer)  [(None, 4)]   0
_____________________________________________________________________
fc_1 (Dense)               (None, 256)   1280     observations[0][0]
_____________________________________________________________________
fc_value_1 (Dense)         (None, 256)   1280     observations[0][0]
_____________________________________________________________________
fc_2 (Dense)               (None, 256)   65792    fc_1[0][0]
_____________________________________________________________________
fc_value_2 (Dense)         (None, 256)   65792    fc_value_1[0][0]
_____________________________________________________________________
fc_out (Dense)             (None, 2)     514      fc_2[0][0]
_____________________________________________________________________
value_out (Dense)          (None, 1)     257      fc_value_2[0][0]
=====================================================================
Total params: 134,915
Trainable params: 134,915
Non-trainable params: 0
_____________________________________________________________________
"""

FullyConnectedNetwork_as_DQNTorchModel(
  (_hidden_layers): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=4, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (advantage_module): Sequential(
    (dueling_A_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
    )
    (A): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=2, bias=True)
      )
    )
  )
  (value_module): Sequential(
    (dueling_V_0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
    )
    (V): S

c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\rllib\algorithms\algorithm.py:568: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may 

'\nModel: "model"\n_____________________________________________________________________\nLayer (type)               Output Shape  Param #  Connected to\n=====================================================================\nobservations (InputLayer)  [(None, 4)]   0\n_____________________________________________________________________\nfc_1 (Dense)               (None, 256)   1280     observations[0][0]\n_____________________________________________________________________\nfc_value_1 (Dense)         (None, 256)   1280     observations[0][0]\n_____________________________________________________________________\nfc_2 (Dense)               (None, 256)   65792    fc_1[0][0]\n_____________________________________________________________________\nfc_value_2 (Dense)         (None, 256)   65792    fc_value_1[0][0]\n_____________________________________________________________________\nfc_out (Dense)             (None, 2)     514      fc_2[0][0]\n____________________________________________